In [1]:
!pip install torch torchvision torchaudio
!pip install ogb
!pip install torch-geometric

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 101.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 71.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitl

In [ ]:
import os, random, numpy as np, torch
import torch.nn.functional as F
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.utils import to_networkx
import networkx as nx
import wandb

wandb.login(key="Your-api-key")
Dataset_name = "IMDB-BINARY"

# *Baseline*

In [3]:
import os
import random
import numpy as np
import torch
import torch.nn.functional as F
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
import wandb

EPOCHS = int(os.getenv("EPOCHS", "50"))
PROJECT = f"GCN Graph classification({Dataset_name})_Gal"
ENTITY = "noynetanel1-tel-aviv-university"

class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.lin = torch.nn.Linear(hidden_channels, out_channels)
    def forward(self, x, edge_index, batch):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = global_mean_pool(x, batch)
        x = self.lin(x)
        return F.log_softmax(x, dim=1)

def set_seed(s):
    random.seed(s)
    np.random.seed(s)
    torch.manual_seed(s)
    torch.cuda.manual_seed_all(s)

def run_seed(seed):
    set_seed(seed)
    
    dataset = TUDataset(root='data/TUDataset', name=Dataset_name)

    # Filter out None graphs and create x if missing
    clean_dataset = []
    for data in dataset:
        if data is None:
            continue
        if data.x is None:
            data.x = torch.ones((data.num_nodes, 1), dtype=torch.float)
        clean_dataset.append(data)

    if len(clean_dataset) == 0:
        raise RuntimeError(f"No valid graphs in dataset. Check your {Dataset_name} installation.")

    dataset = clean_dataset

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    in_channels = dataset[0].x.shape[1]
    out_channels = max([data.y.max().item() for data in dataset]) + 1
    model = GCN(in_channels, 64, out_channels).to(device)

    # Shuffle using random.shuffle instead of dataset.shuffle()
    random.shuffle(dataset)

    num_graphs = len(dataset)
    train_size = int(0.8 * num_graphs)
    train_dataset = dataset[:train_size]
    test_dataset = dataset[train_size:]
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    wandb.init(
        project=PROJECT,
        entity=ENTITY,
        name=f"GCN_baseline_seed_{seed}",
        group=f"GCN_{Dataset_name}_baseline",
        job_type="baseline",
        reinit=True,
        config={
            "model":"GCN",
            "dataset":Dataset_name,
            "hidden_channels":64,
            "lr":0.001,
            "epochs":EPOCHS,
            "task":"Baseline",
            "seed":seed
        }
    )

    def train():
        model.train()
        total_loss = 0.0
        for batch in train_loader:
            batch = batch.to(device)
            optimizer.zero_grad()
            out = model(batch.x, batch.edge_index, batch.batch)
            loss = F.nll_loss(out, batch.y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        return total_loss / len(train_loader)

    def test(loader):
        model.eval()
        correct = 0
        for batch in loader:
            batch = batch.to(device)
            out = model(batch.x, batch.edge_index, batch.batch)
            pred = out.argmax(dim=1)
            correct += (pred == batch.y).sum().item()
        return correct / len(loader.dataset)

    for epoch in range(1, EPOCHS + 1):
        loss = train()
        test_acc = test(test_loader)
        train_acc = test(train_loader)
        
        print(f"Seed {seed} | Epoch {epoch:03d} | Loss: {loss:.4f} | Test Acc: {test_acc:.4f}")
        wandb.log({"epoch": epoch, "loss": loss, "train_acc": train_acc, "test_acc": test_acc})

    wandb.finish()

if __name__ == "__main__":
    for s in range(1, 31):
        run_seed(s)


Processing...
Done!
wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.
wandb: Tracking run with wandb version 0.20.1
wandb: Run data is saved locally in /kaggle/working/wandb/run-20250822_131915-f1gu314d
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run GCN_baseline_seed_1
wandb: ⭐️ View project at https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28IMDB-BINARY%29_Gal
wandb: 🚀 View run at https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28IMDB-BINARY%29_Gal/runs/f1gu314d


Seed 1 | Epoch 001 | Loss: 0.6950 | Test Acc: 0.4600
Seed 1 | Epoch 002 | Loss: 0.6941 | Test Acc: 0.4600
Seed 1 | Epoch 003 | Loss: 0.6941 | Test Acc: 0.4600
Seed 1 | Epoch 004 | Loss: 0.6933 | Test Acc: 0.4600
Seed 1 | Epoch 005 | Loss: 0.6949 | Test Acc: 0.4600
Seed 1 | Epoch 006 | Loss: 0.6945 | Test Acc: 0.4600
Seed 1 | Epoch 007 | Loss: 0.6943 | Test Acc: 0.4600
Seed 1 | Epoch 008 | Loss: 0.6942 | Test Acc: 0.4600
Seed 1 | Epoch 009 | Loss: 0.6932 | Test Acc: 0.4600
Seed 1 | Epoch 010 | Loss: 0.6935 | Test Acc: 0.4600
Seed 1 | Epoch 011 | Loss: 0.6943 | Test Acc: 0.4600
Seed 1 | Epoch 012 | Loss: 0.6930 | Test Acc: 0.4600
Seed 1 | Epoch 013 | Loss: 0.6932 | Test Acc: 0.4600
Seed 1 | Epoch 014 | Loss: 0.6931 | Test Acc: 0.4600
Seed 1 | Epoch 015 | Loss: 0.6931 | Test Acc: 0.4600
Seed 1 | Epoch 016 | Loss: 0.6932 | Test Acc: 0.4600
Seed 1 | Epoch 017 | Loss: 0.6935 | Test Acc: 0.4600
Seed 1 | Epoch 018 | Loss: 0.6934 | Test Acc: 0.4600
Seed 1 | Epoch 019 | Loss: 0.6930 | Test Acc: 

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇██
wandb:      loss █▆▆█▇▆▃▄▆▃▃▃▃▄▄▄▃▃▃▄▃▄▃▃▄▄▂▆▂▃▂▂▂▂▂▂▃▂▂▁
wandb:  test_acc ▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▇▁▁▁▆▁
wandb: train_acc ▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▅▂▂▂▂█▂▁▁▇▂
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 0.69242
wandb:  test_acc 0.46
wandb: train_acc 0.51
wandb: 
wandb: 🚀 View run GCN_baseline_seed_1 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28IMDB-BINARY%29_Gal/runs/f1gu314d
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28IMDB-BINARY%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250822_131915-f1gu314d/logs
wandb: Tracking run with wandb version 0.20.1
wandb: Run data is saved locally in /kaggle/working/w

Seed 2 | Epoch 001 | Loss: 0.6946 | Test Acc: 0.4450
Seed 2 | Epoch 002 | Loss: 0.6943 | Test Acc: 0.4450
Seed 2 | Epoch 003 | Loss: 0.6949 | Test Acc: 0.4450
Seed 2 | Epoch 004 | Loss: 0.6941 | Test Acc: 0.5550
Seed 2 | Epoch 005 | Loss: 0.6930 | Test Acc: 0.4450
Seed 2 | Epoch 006 | Loss: 0.6933 | Test Acc: 0.4450
Seed 2 | Epoch 007 | Loss: 0.6934 | Test Acc: 0.4450
Seed 2 | Epoch 008 | Loss: 0.6930 | Test Acc: 0.4450
Seed 2 | Epoch 009 | Loss: 0.6937 | Test Acc: 0.4450
Seed 2 | Epoch 010 | Loss: 0.6930 | Test Acc: 0.4450
Seed 2 | Epoch 011 | Loss: 0.6932 | Test Acc: 0.4450
Seed 2 | Epoch 012 | Loss: 0.6931 | Test Acc: 0.4450
Seed 2 | Epoch 013 | Loss: 0.6937 | Test Acc: 0.4450
Seed 2 | Epoch 014 | Loss: 0.6929 | Test Acc: 0.4450
Seed 2 | Epoch 015 | Loss: 0.6931 | Test Acc: 0.4450
Seed 2 | Epoch 016 | Loss: 0.6928 | Test Acc: 0.4450
Seed 2 | Epoch 017 | Loss: 0.6933 | Test Acc: 0.4450
Seed 2 | Epoch 018 | Loss: 0.6939 | Test Acc: 0.4450
Seed 2 | Epoch 019 | Loss: 0.6933 | Test Acc: 

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
wandb:      loss ▇▆█▅▂▂▄▂▂▂▁▂▁▃▅▁▂▄▁▂▂▂▂▃▃▃▂▂▂▃▁▂▁▂▂▃▂▁▅▁
wandb:  test_acc ▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb: train_acc ███▁████████████████████████████████████
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 0.69293
wandb:  test_acc 0.445
wandb: train_acc 0.51375
wandb: 
wandb: 🚀 View run GCN_baseline_seed_2 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28IMDB-BINARY%29_Gal/runs/inrvhmx1
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28IMDB-BINARY%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250822_131936-inrvhmx1/logs
wandb: Tracking run with wandb version 0.20.1
wandb: Run data is saved locally in /kaggle/worki

...

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
wandb:      loss █▆▅▄▅▃▃▅▂▃▅▁▃▅▂▅█▄▂▂▆▃▃▂▃▄▃▅▄▂▄▂▁▃▃▄▂▂▁▃
wandb:  test_acc ▁▁▁▂▂▂▂▁▂▁▁▂▂▂▁▂▁▂▁▁▂▁▁█▁▁▁▁▁▁▂▁▁▂▁▁▁▁▃▁
wandb: train_acc ▂▂▂▁▁▁▁▂▁▂▂▁▁▁▂▁▂▁▂▂▂▂█▂▂▂▁▂▂▂▁▂▂▁▁▂▂▂▃▂
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 0.69303
wandb:  test_acc 0.495
wandb: train_acc 0.50125
wandb: 
wandb: 🚀 View run GCN_baseline_seed_28 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28IMDB-BINARY%29_Gal/runs/ef5e9i8w
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28IMDB-BINARY%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250822_132824-ef5e9i8w/logs
wandb: Tracking run with wandb version 0.20.1
wandb: Run data is saved locally in /kaggle/work

Seed 29 | Epoch 001 | Loss: 0.6946 | Test Acc: 0.5050
Seed 29 | Epoch 002 | Loss: 0.6940 | Test Acc: 0.4950
Seed 29 | Epoch 003 | Loss: 0.6948 | Test Acc: 0.4950
Seed 29 | Epoch 004 | Loss: 0.6942 | Test Acc: 0.4950
Seed 29 | Epoch 005 | Loss: 0.6937 | Test Acc: 0.5050
Seed 29 | Epoch 006 | Loss: 0.6939 | Test Acc: 0.4950
Seed 29 | Epoch 007 | Loss: 0.6934 | Test Acc: 0.5050
Seed 29 | Epoch 008 | Loss: 0.6938 | Test Acc: 0.4950
Seed 29 | Epoch 009 | Loss: 0.6941 | Test Acc: 0.4950
Seed 29 | Epoch 010 | Loss: 0.6934 | Test Acc: 0.4950
Seed 29 | Epoch 011 | Loss: 0.6939 | Test Acc: 0.5050
Seed 29 | Epoch 012 | Loss: 0.6934 | Test Acc: 0.4950
Seed 29 | Epoch 013 | Loss: 0.6933 | Test Acc: 0.5050
Seed 29 | Epoch 014 | Loss: 0.6934 | Test Acc: 0.5050
Seed 29 | Epoch 015 | Loss: 0.6944 | Test Acc: 0.5050
Seed 29 | Epoch 016 | Loss: 0.6955 | Test Acc: 0.4950
Seed 29 | Epoch 017 | Loss: 0.6934 | Test Acc: 0.5050
Seed 29 | Epoch 018 | Loss: 0.6936 | Test Acc: 0.5050
Seed 29 | Epoch 019 | Loss: 

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
wandb:      loss ▅▃▆▄▃▂▃▄▂▃▁▂▅█▂▃▂▁▄▃▁▂▁▁▃▁▁▂▁▂▂▁▂▁▁▃▂▁▂▁
wandb:  test_acc ▃▁▁▁▃▃▁▁▁▃▃▃▃▁▃▃▃▁▃▁▃▁▁▃▁▁▁▃▁▁▁▁█▃▁▁▃▁▁▁
wandb: train_acc ▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁█▁▂▂▂▂▂▂
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 0.69319
wandb:  test_acc 0.495
wandb: train_acc 0.50125
wandb: 
wandb: 🚀 View run GCN_baseline_seed_29 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28IMDB-BINARY%29_Gal/runs/mvfigej8
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28IMDB-BINARY%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250822_132843-mvfigej8/logs
wandb: Tracking run with wandb version 0.20.1
wandb: Run data is saved locally in /kaggle/work

Seed 30 | Epoch 001 | Loss: 0.7015 | Test Acc: 0.4850
Seed 30 | Epoch 002 | Loss: 0.6946 | Test Acc: 0.4150
Seed 30 | Epoch 003 | Loss: 0.6947 | Test Acc: 0.4850
Seed 30 | Epoch 004 | Loss: 0.6962 | Test Acc: 0.4850
Seed 30 | Epoch 005 | Loss: 0.6948 | Test Acc: 0.4850
Seed 30 | Epoch 006 | Loss: 0.6939 | Test Acc: 0.4850
Seed 30 | Epoch 007 | Loss: 0.6940 | Test Acc: 0.5150
Seed 30 | Epoch 008 | Loss: 0.6939 | Test Acc: 0.4850
Seed 30 | Epoch 009 | Loss: 0.6936 | Test Acc: 0.4850
Seed 30 | Epoch 010 | Loss: 0.6948 | Test Acc: 0.4850
Seed 30 | Epoch 011 | Loss: 0.6953 | Test Acc: 0.5150
Seed 30 | Epoch 012 | Loss: 0.6947 | Test Acc: 0.4850
Seed 30 | Epoch 013 | Loss: 0.6936 | Test Acc: 0.5150
Seed 30 | Epoch 014 | Loss: 0.6935 | Test Acc: 0.4850
Seed 30 | Epoch 015 | Loss: 0.6947 | Test Acc: 0.4850
Seed 30 | Epoch 016 | Loss: 0.6941 | Test Acc: 0.4850
Seed 30 | Epoch 017 | Loss: 0.6939 | Test Acc: 0.4850
Seed 30 | Epoch 018 | Loss: 0.6935 | Test Acc: 0.4850
Seed 30 | Epoch 019 | Loss: 

wandb: uploading output.log
wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
wandb:      loss █▂▂▃▂▁▁▁▂▃▁▁▂▂▁▁▁▁▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:  test_acc ▆▂▆▆▆▆▆▆█▆▆▆▆▆▆▆▆▆▆▆█▆▆▆▆▆▁▆█▆▆▆▆▆▆▆▆▆█▆
wandb: train_acc ████████▇████████████▇█████▁██████████▇█
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 0.69358
wandb:  test_acc 0.485
wandb: train_acc 0.50375
wandb: 
wandb: 🚀 View run GCN_baseline_seed_30 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28IMDB-BINARY%29_Gal/runs/fb1hmqde
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28IMDB-BINARY%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250822_132904-fb1hmqde/logs


# *BFS*

In [4]:
import os, random, numpy as np, torch
import torch.nn.functional as F
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.utils import to_networkx
import networkx as nx
import wandb

EPOCHS = int(os.getenv("EPOCHS", "50"))
NUM_SEEDS = int(os.getenv("NUM_SEEDS", "30"))
PROJECT = f"GCN Graph classification({Dataset_name})_Gal"
ENTITY = "noynetanel1-tel-aviv-university"

class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.lin = torch.nn.Linear(hidden_channels, out_channels)

    def forward(self, x, edge_index, batch):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = global_mean_pool(x, batch)
        return F.log_softmax(self.lin(x), dim=1)

def set_seed(s):
    random.seed(s)
    np.random.seed(s)
    torch.manual_seed(s)
    torch.cuda.manual_seed_all(s)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def add_bfs_rank(d):
    d = d.clone()
    if d.x is None:
        d.x = torch.ones((d.num_nodes, 1), dtype=torch.float)
    G = to_networkx(d, to_undirected=True)
    root = int(d.edge_index[0][0].item()) if d.edge_index.numel() > 0 else 0
    bfs_order = list(nx.bfs_tree(G, root)) if G.number_of_nodes() > 0 else []
    bfs_rank = torch.full((d.num_nodes, 1), -1.0, dtype=torch.float)
    L = max(1, len(bfs_order))
    for i, node in enumerate(bfs_order):
        bfs_rank[node] = float(i) / float(L)
    d.x = torch.cat([d.x, bfs_rank], dim=1)
    return d

def run_seed(seed):
    set_seed(seed)
    dataset = TUDataset(root='data/TUDataset', name=Dataset_name, transform=add_bfs_rank)
    torch.manual_seed(seed)
    dataset = dataset.shuffle()
    num_graphs = len(dataset)
    train_size = int(0.8 * num_graphs)
    train_dataset = dataset[:train_size]
    test_dataset = dataset[train_size:]
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    in_channels = dataset[0].x.size(1)
    model = GCN(in_channels, 64, dataset.num_classes).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    wandb.init(
        project=PROJECT,
        entity=ENTITY,
        name=f"GCN_BFS_seed_{seed}",
        group=f"GCN_{Dataset_name}_BFS",
        job_type="bfs",
        reinit=True,
        config={"model":"GCN","hidden_channels":64,"epochs":EPOCHS,"lr":0.001,"dataset":"REDDIT-BINARY","task":"BFS","positional_encoding":"BFS","seed":seed}
    )

    def train():
        model.train()
        total_loss = 0.0
        for data in train_loader:
            data = data.to(device)
            optimizer.zero_grad()
            out = model(data.x, data.edge_index, data.batch)
            loss = F.nll_loss(out, data.y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        return total_loss / len(train_loader)

    def test(loader):
        model.eval()
        correct = 0
        for data in loader:
            data = data.to(device)
            out = model(data.x, data.edge_index, data.batch)
            pred = out.argmax(dim=1)
            correct += int((pred == data.y).sum())
        return correct / len(loader.dataset)

    for epoch in range(1, EPOCHS + 1):
        loss = train()
        test_acc = test(test_loader)
        train_acc = test(train_loader)
        print(f"Seed {seed} | Epoch {epoch:03d} | Loss: {loss:.4f} | Test Acc: {test_acc:.4f}")
        wandb.log({"epoch": epoch, "loss": loss, "train_acc": train_acc, "test_acc": test_acc})
    wandb.finish()

if __name__ == "__main__":
    wandb.login()
    for s in range(1, NUM_SEEDS + 1):
        run_seed(s)


wandb: Tracking run with wandb version 0.20.1
wandb: Run data is saved locally in /kaggle/working/wandb/run-20250822_132926-nqc1v9r0
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run GCN_BFS_seed_1
wandb: ⭐️ View project at https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28IMDB-BINARY%29_Gal
wandb: 🚀 View run at https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28IMDB-BINARY%29_Gal/runs/nqc1v9r0


Seed 1 | Epoch 001 | Loss: 0.6966 | Test Acc: 0.4750
Seed 1 | Epoch 002 | Loss: 0.6941 | Test Acc: 0.4200
Seed 1 | Epoch 003 | Loss: 0.6936 | Test Acc: 0.4750
Seed 1 | Epoch 004 | Loss: 0.6937 | Test Acc: 0.4750
Seed 1 | Epoch 005 | Loss: 0.6933 | Test Acc: 0.5250
Seed 1 | Epoch 006 | Loss: 0.6940 | Test Acc: 0.4750
Seed 1 | Epoch 007 | Loss: 0.6937 | Test Acc: 0.4750
Seed 1 | Epoch 008 | Loss: 0.6935 | Test Acc: 0.5250
Seed 1 | Epoch 009 | Loss: 0.6937 | Test Acc: 0.4750
Seed 1 | Epoch 010 | Loss: 0.6939 | Test Acc: 0.5150
Seed 1 | Epoch 011 | Loss: 0.6934 | Test Acc: 0.4750
Seed 1 | Epoch 012 | Loss: 0.6933 | Test Acc: 0.4750
Seed 1 | Epoch 013 | Loss: 0.6931 | Test Acc: 0.4750
Seed 1 | Epoch 014 | Loss: 0.6943 | Test Acc: 0.4750
Seed 1 | Epoch 015 | Loss: 0.6939 | Test Acc: 0.4750
Seed 1 | Epoch 016 | Loss: 0.6936 | Test Acc: 0.4750
Seed 1 | Epoch 017 | Loss: 0.6941 | Test Acc: 0.5250
Seed 1 | Epoch 018 | Loss: 0.6932 | Test Acc: 0.4750
Seed 1 | Epoch 019 | Loss: 0.6938 | Test Acc: 

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
wandb:      loss █▆▆▆▅▆▆▆▆▆▅▆▆▆▆▆▆▅▆▆▅▅▅▆▅▅▄▄▅▅▃▃▃▃▃▂▂▁▁▁
wandb:  test_acc ▃▁▃▃▄▃▄▃▄▃▃▃▃▃▄▄▃▃▃▃▃▃▃▇▆▃▃▇▆▇▆▃▇▇▆▆██▇█
wandb: train_acc ▃▁▃▃▂▃▂▃▁▃▃▃▃▃▂▃▃▃▃▇▃▃▆▆▃▃▇▆▃▅▆▃▇▇▇▇▅██▇
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 0.6879
wandb:  test_acc 0.64
wandb: train_acc 0.57875
wandb: 
wandb: 🚀 View run GCN_BFS_seed_1 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28IMDB-BINARY%29_Gal/runs/nqc1v9r0
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28IMDB-BINARY%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250822_132926-nqc1v9r0/logs
wandb: Tracking run with wandb version 0.20.1
wandb: Run data is saved locally in /kaggle/working/wand

Seed 2 | Epoch 001 | Loss: 0.6941 | Test Acc: 0.5250
Seed 2 | Epoch 002 | Loss: 0.6942 | Test Acc: 0.4750
Seed 2 | Epoch 003 | Loss: 0.6954 | Test Acc: 0.4750
Seed 2 | Epoch 004 | Loss: 0.6947 | Test Acc: 0.4750
Seed 2 | Epoch 005 | Loss: 0.6949 | Test Acc: 0.5250
Seed 2 | Epoch 006 | Loss: 0.6952 | Test Acc: 0.4750
Seed 2 | Epoch 007 | Loss: 0.6931 | Test Acc: 0.4750
Seed 2 | Epoch 008 | Loss: 0.6932 | Test Acc: 0.4750
Seed 2 | Epoch 009 | Loss: 0.6933 | Test Acc: 0.4750
Seed 2 | Epoch 010 | Loss: 0.6932 | Test Acc: 0.4750
Seed 2 | Epoch 011 | Loss: 0.6930 | Test Acc: 0.4750
Seed 2 | Epoch 012 | Loss: 0.6933 | Test Acc: 0.5900
Seed 2 | Epoch 013 | Loss: 0.6928 | Test Acc: 0.4750
Seed 2 | Epoch 014 | Loss: 0.6927 | Test Acc: 0.4750
Seed 2 | Epoch 015 | Loss: 0.6930 | Test Acc: 0.6200
Seed 2 | Epoch 016 | Loss: 0.6927 | Test Acc: 0.4750
Seed 2 | Epoch 017 | Loss: 0.6931 | Test Acc: 0.4800
Seed 2 | Epoch 018 | Loss: 0.6933 | Test Acc: 0.4750
Seed 2 | Epoch 019 | Loss: 0.6924 | Test Acc: 

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
wandb:      loss ▇▇███▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▃▃▄▂▂▂▁▁
wandb:  test_acc ▃▁▁▁▃▁▁▁▁▁▁▁█▁▁█▁█▃▁▁▅▇▅▅▇▆▆▆▆▆▆▆▇▇▇▆▆▆▆
wandb: train_acc ▃▃▃▂▃▃▃▃▃▅▃▅▃▃▃▃▅▁▃▄▅▅▅▆▆▄▇▆▇▅▆▇▅▇▆▇█▆█▇
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 0.67973
wandb:  test_acc 0.59
wandb: train_acc 0.58625
wandb: 
wandb: 🚀 View run GCN_BFS_seed_2 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28IMDB-BINARY%29_Gal/runs/yjj5wtir
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28IMDB-BINARY%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250822_133108-yjj5wtir/logs
wandb: Tracking run with wandb version 0.20.1
wandb: Run data is saved locally in /kaggle/working/wan

...

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
wandb:      loss █▅▅▅▅▄▄▅▄▄▄▄▅▄▄▄▄▄▄▄▃▄▃▃▃▄▄▃▃▃▂▂▂▂▂▁▃▂▂▂
wandb:  test_acc ▄▁▁▁▁▁▁▁▁▄▁▆▁▁▁▁▃▁▁▁▅▁▄▁▇▇▅▅▆▇▇▇▆▆▇▆▆▅▆█
wandb: train_acc ▁▂▂▂▁▂▂▄▂▂▂▂▂▄▂▂▂▂▂▆▄▂▅▆▅▅▅▅▃▇▇▅▅▆█▅▅▅██
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 0.68887
wandb:  test_acc 0.59
wandb: train_acc 0.585
wandb: 
wandb: 🚀 View run GCN_BFS_seed_28 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28IMDB-BINARY%29_Gal/runs/0c5itf9m
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28IMDB-BINARY%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250822_141331-0c5itf9m/logs
wandb: Tracking run with wandb version 0.20.1
wandb: Run data is saved locally in /kaggle/working/wand

Seed 29 | Epoch 001 | Loss: 0.6968 | Test Acc: 0.5150
Seed 29 | Epoch 002 | Loss: 0.6932 | Test Acc: 0.4850
Seed 29 | Epoch 003 | Loss: 0.6943 | Test Acc: 0.5150
Seed 29 | Epoch 004 | Loss: 0.6946 | Test Acc: 0.5150
Seed 29 | Epoch 005 | Loss: 0.6939 | Test Acc: 0.4850
Seed 29 | Epoch 006 | Loss: 0.6932 | Test Acc: 0.4850
Seed 29 | Epoch 007 | Loss: 0.6936 | Test Acc: 0.4850
Seed 29 | Epoch 008 | Loss: 0.6930 | Test Acc: 0.5450
Seed 29 | Epoch 009 | Loss: 0.6931 | Test Acc: 0.4850
Seed 29 | Epoch 010 | Loss: 0.6928 | Test Acc: 0.5450
Seed 29 | Epoch 011 | Loss: 0.6924 | Test Acc: 0.4850
Seed 29 | Epoch 012 | Loss: 0.6924 | Test Acc: 0.5200
Seed 29 | Epoch 013 | Loss: 0.6924 | Test Acc: 0.5500
Seed 29 | Epoch 014 | Loss: 0.6922 | Test Acc: 0.5300
Seed 29 | Epoch 015 | Loss: 0.6915 | Test Acc: 0.5600
Seed 29 | Epoch 016 | Loss: 0.6919 | Test Acc: 0.4850
Seed 29 | Epoch 017 | Loss: 0.6913 | Test Acc: 0.4650
Seed 29 | Epoch 018 | Loss: 0.6927 | Test Acc: 0.5050
Seed 29 | Epoch 019 | Loss: 

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
wandb:      loss █▆▇▇▇▇▆▆▆▆▆▆▆▆▆▆▅▅▅▅▅▅▅▄▄▃▃▄▄▃▂▂▂▂▁▁▃▁▁▃
wandb:  test_acc ▃▂▃▃▂▂▅▂▅▂▅▄▆▂▁▃▄▄▅▄▃▅▄▅▅▅▄▆▅▆▅▅▅▅██▅▆▅▇
wandb: train_acc ▁▂▁▂▂▅▂▄▂▅▆▅▂▃▃▅▄▅▆▆▄▅▅▆▆▆▅▇▇▇▇▅▇▆▇█▆█▇█
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 0.68424
wandb:  test_acc 0.59
wandb: train_acc 0.6
wandb: 
wandb: 🚀 View run GCN_BFS_seed_29 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28IMDB-BINARY%29_Gal/runs/w8e4415e
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28IMDB-BINARY%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250822_141507-w8e4415e/logs
wandb: Tracking run with wandb version 0.20.1
wandb: Run data is saved locally in /kaggle/working/wandb/

Seed 30 | Epoch 001 | Loss: 0.6973 | Test Acc: 0.4350
Seed 30 | Epoch 002 | Loss: 0.6951 | Test Acc: 0.5650
Seed 30 | Epoch 003 | Loss: 0.6948 | Test Acc: 0.4350
Seed 30 | Epoch 004 | Loss: 0.6925 | Test Acc: 0.4350
Seed 30 | Epoch 005 | Loss: 0.6929 | Test Acc: 0.4350
Seed 30 | Epoch 006 | Loss: 0.6927 | Test Acc: 0.4350
Seed 30 | Epoch 007 | Loss: 0.6934 | Test Acc: 0.4350
Seed 30 | Epoch 008 | Loss: 0.6927 | Test Acc: 0.4350
Seed 30 | Epoch 009 | Loss: 0.6942 | Test Acc: 0.4350
Seed 30 | Epoch 010 | Loss: 0.6936 | Test Acc: 0.4350
Seed 30 | Epoch 011 | Loss: 0.6939 | Test Acc: 0.4350
Seed 30 | Epoch 012 | Loss: 0.6922 | Test Acc: 0.4350
Seed 30 | Epoch 013 | Loss: 0.6921 | Test Acc: 0.4350
Seed 30 | Epoch 014 | Loss: 0.6926 | Test Acc: 0.4350
Seed 30 | Epoch 015 | Loss: 0.6924 | Test Acc: 0.4350
Seed 30 | Epoch 016 | Loss: 0.6931 | Test Acc: 0.4350
Seed 30 | Epoch 017 | Loss: 0.6917 | Test Acc: 0.4350
Seed 30 | Epoch 018 | Loss: 0.6918 | Test Acc: 0.4350
Seed 30 | Epoch 019 | Loss: 

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██
wandb:      loss █▇▇▆▆▆▆▇▆▇▆▆▆▆▅▅▅▅▅▅▅▅▄▅▅▄▄▅▄▄▃▃▃▂▂▁▂▂▂▁
wandb:  test_acc ▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▆▇▆▁▄██▇▁▇▇▇▁▇▁█
wandb: train_acc ▃▁▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▅▃▃▃▅▅▆▃▅▆▅▇▃▇▇▄▃▅▃█
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 0.68167
wandb:  test_acc 0.605
wandb: train_acc 0.60625
wandb: 
wandb: 🚀 View run GCN_BFS_seed_30 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28IMDB-BINARY%29_Gal/runs/raggeh20
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28IMDB-BINARY%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250822_141643-raggeh20/logs
